<a href="https://colab.research.google.com/github/ernesto-miguez/Machine-Learning-training/blob/master/Hopular_vs_HGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder

In [ ]:
url = 'https://raw.githubusercontent.com/tunguz/TabularBenchmarks/main/input/colleges/colleges.csv'
colleges = pd.read_csv(url, index_col=0)

In [ ]:
colleges.head()

,school_name,city,state,zip,school_webpage,latitude,longitude,admission_rate,sat_verbal_midrange,sat_math_midrange,...,carnegie_undergraduate,carnegie_size,religious_affiliation,percent_female,agege24,faminc,mean_earnings_6_years,median_earnings_6_years,mean_earnings_10_years,median_earnings_10_years
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,'Alabama A & M University',Normal,AL,35762,www.aamu.edu/,34.7834,-86.5685,0.8989,410.0,400.0,...,'Full-time four-year inclusive','Medium 4-year highly residential (3000 to 9999)',?,0.52999997138977,0.07999999821186,40211.22,26100.0,22800.0,35300.0,31400.0
100663,'University of Alabama at Birmingham',Birmingham,AL,35294-0110,www.uab.edu,33.5022,-86.8092,0.8673,580.0,585.0,...,'Medium full-time four-year selective higher t...,'Large 4-year primarily nonresidential (over 9...,?,0.64999997615814,0.25999999046325,49894.65,37400.0,33200.0,46300.0,40300.0
100690,'Amridge University',Montgomery,AL,36117-3553,www.amridgeuniversity.edu,32.3626,-86.17399999999999,?,?,?,...,'Medium full-time four-year inclusivestudents ...,'Very small 4-year primarily nonresidential (l...,'Churches of Christ',0.50999999046325,0.82999998331069,38712.18,38500.0,32800.0,42100.0,38100.0
100706,'University of Alabama in Huntsville',Huntsville,AL,35899,www.uah.edu,34.7228,-86.6384,0.8062,575.0,580.0,...,'Medium full-time four-year selective higher t...,'Medium 4-year primarily nonresidential (3000 ...,?,0.55000001192092,0.28999999165534,54155.4,39300.0,36700.0,52700.0,46600.0
100724,'Alabama State University',Montgomery,AL,36104-0271,www.alasu.edu/email/index.aspx,32.3643,-86.2957,0.5125,430.0,425.0,...,'Full-time four-year inclusive','Medium 4-year primarily residential (3000 to ...,?,0.56999999284744,0.10999999940395,31846.99,21200.0,19300.0,30700.0,27800.0


In [ ]:
colleges.columns

Index(['school_name', 'city', 'state', 'zip', 'school_webpage', 'latitude',
       'longitude', 'admission_rate', 'sat_verbal_midrange',
       'sat_math_midrange', 'sat_writing_midrange', 'act_combined_midrange',
       'act_english_midrange', 'act_math_midrange', 'act_writing_midrange',
       'sat_total_average', 'undergrad_size', 'percent_white', 'percent_black',
       'percent_hispanic', 'percent_asian', 'percent_part_time',
       'average_cost_academic_year', 'average_cost_program_year',
       'tuition_(instate)', 'tuition_(out_of_state)', 'spend_per_student',
       'faculty_salary', 'percent_part_time_faculty', 'percent_pell_grant',
       'completion_rate', 'predominant_degree', 'highest_degree', 'ownership',
       'region', 'gender', 'carnegie_basic_classification',
       'carnegie_undergraduate', 'carnegie_size', 'religious_affiliation',
       'percent_female', 'agege24', 'faminc', 'mean_earnings_6_years',
       'median_earnings_6_years', 'mean_earnings_10_years',
   

In [ ]:
colleges.shape

(7063, 47)

In [ ]:
colleges['admission_rate'] = pd.to_numeric(colleges['admission_rate'], errors='coerce')
colleges['undergrad_size'] = pd.to_numeric(colleges['undergrad_size'], errors='coerce')
colleges['sat_math_midrange'] = pd.to_numeric(colleges['sat_math_midrange'], errors='coerce')
colleges['sat_verbal_midrange'] = pd.to_numeric(colleges['sat_verbal_midrange'], errors='coerce')
colleges['act_writing_midrange'] = pd.to_numeric(colleges['act_writing_midrange'], errors='coerce')
colleges['act_math_midrange'] = pd.to_numeric(colleges['act_math_midrange'], errors='coerce')
colleges['act_english_midrange'] = pd.to_numeric(colleges['act_english_midrange'], errors='coerce')
colleges['act_combined_midrange'] = pd.to_numeric(colleges['act_combined_midrange'], errors='coerce')
colleges['completion_rate'] = pd.to_numeric(colleges['completion_rate'], errors='coerce') 
colleges['tuition_(instate)'] = pd.to_numeric(colleges['tuition_(instate)'], errors='coerce')
colleges['tuition_(out_of_state)'] = pd.to_numeric(colleges['tuition_(out_of_state)'], errors='coerce')
colleges['percent_white'] = pd.to_numeric(colleges['percent_white'], errors='coerce')
colleges['percent_black'] = pd.to_numeric(colleges['percent_black'], errors='coerce')
colleges['percent_hispanic'] = pd.to_numeric(colleges['percent_hispanic'], errors='coerce')
colleges['percent_asian'] = pd.to_numeric(colleges['percent_asian'], errors='coerce')
colleges['sat_total_average'] = pd.to_numeric(colleges['sat_total_average'], errors='coerce')
colleges['completion_rate'] = pd.to_numeric(colleges['completion_rate'], errors='coerce')
colleges['percent_female'] = pd.to_numeric(colleges['percent_female'], errors='coerce')
colleges['agege24'] = pd.to_numeric(colleges['agege24'], errors='coerce')
colleges['faculty_salary'] = pd.to_numeric(colleges['faculty_salary'], errors='coerce')
colleges['percent_part_time'] = pd.to_numeric(colleges['percent_part_time'], errors='coerce')
colleges['faminc'] = pd.to_numeric(colleges['faminc'], errors='coerce')
colleges['average_cost_academic_year'] = pd.to_numeric(colleges['average_cost_academic_year'], errors='coerce')
colleges['average_cost_program_year'] = pd.to_numeric(colleges['average_cost_program_year'], errors='coerce')
colleges['spend_per_student'] = pd.to_numeric(colleges['spend_per_student'], errors='coerce')
colleges['mean_earnings_6_years'] = pd.to_numeric(colleges['mean_earnings_6_years'], errors='coerce')
colleges['median_earnings_6_years'] = pd.to_numeric(colleges['median_earnings_6_years'], errors='coerce')
colleges['mean_earnings_10_years'] = pd.to_numeric(colleges['mean_earnings_10_years'], errors='coerce')
colleges['median_earnings_10_years'] = pd.to_numeric(colleges['median_earnings_10_years'], errors='coerce')
colleges['latitude'] = pd.to_numeric(colleges['latitude'], errors='coerce')
colleges['longitude'] = pd.to_numeric(colleges['longitude'], errors='coerce')

In [ ]:
columns = ['undergrad_size', 'spend_per_student', 'admission_rate', 'mean_earnings_6_years', 'median_earnings_6_years', 'sat_math_midrange', 'sat_verbal_midrange', 
           'act_math_midrange', 'act_writing_midrange', 'latitude', 'longitude', 'completion_rate', 'tuition_(out_of_state)', 'tuition_(instate)', 'percent_white', 
           'percent_black', 'percent_hispanic', 'percent_asian', 'median_earnings_10_years', 'sat_total_average', 'completion_rate', 'faculty_salary', 'percent_female',
          'percent_part_time', 'agege24', 'faminc', 'average_cost_program_year', 'average_cost_academic_year']

In [ ]:
target = colleges['percent_pell_grant'].values

In [ ]:
errors = []
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in kf.split(colleges):
    X_train, X_test = colleges.iloc[train_index], colleges.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    clf = HistGradientBoostingRegressor()
    clf.fit(X_train[columns], y_train)
    preds = clf.predict(X_test[columns])
    error = mean_squared_error(y_test, preds, squared=True)
    print(f"Fold mean squared error: {error}")
    errors.append(error)

Fold mean squared error: 0.019315167008313667
Fold mean squared error: 0.022547313561870317
Fold mean squared error: 0.02033797505885874
Fold mean squared error: 0.020246076263725377
Fold mean squared error: 0.020241796499142553
